In [13]:
!pip install rank_bm25 nltk transformers pandas numpy nltk chromadb clickhouse_connect torch

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Using cached torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cudnn_

In [14]:
from embedding import E5LargeEmbeddingFunction
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import nltk
from nltk.tokenize import word_tokenize
import clickhouse_connect
import pandas as pd
import num2text
import numpy as np
from rank_bm25 import BM25Okapi
import logging

/home/veidlink/HAKATONS/cb-purple-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/veidlink/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

In [37]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
def get_window_range(num, window_range):
    answer = [num]
    while len(answer) < window_range:
        f = 0
        left = answer[0] - 1
        right = answer[-1] + 1

        if left >= 0:
            answer = [left] + answer
            if len(answer) >= window_range:
                return answer

        answer.append(right)
    
    return answer

In [39]:
def _clickhouse_query_l2(query, client, emb_func, limit=5, table='index_texts_clean'):
    emb_func.change_mode('query')
    embeddings = emb_func(query)[0]

    result = client.query(f'''SELECT
        ID, chunk_id,
        text,
        L2Distance(embedding, {embeddings}) AS score
    FROM {table}
    ORDER BY score ASC
    LIMIT {limit}''')

    return result.result_rows

In [40]:
def _clickhouse_query_window(query, client, emb_func, limit_knn=5, docs_window=5, table='index_texts_clean'):
    res = _clickhouse_query_l2(query, client, emb_func, limit=limit_knn, table=table)

    window_sql_query = ''
    for i, row in enumerate(res):
        #ids_with_chunks[row[0]].extend(get_window_range(row[1], window))
        if i > 0:
            window_sql_query += ' UNION DISTINCT '

        window_sql_query += f'''SELECT * FROM {table} WHERE ID  in {tuple(get_window_range(row[0], docs_window))}'''

    result = client.query(window_sql_query)
    return result.result_rows

In [41]:
model = AutoModelForSequenceClassification.from_pretrained('SkolkovoInstitute/ruRoberta-large-paraphrase-v1')
tokenizer = AutoTokenizer.from_pretrained('SkolkovoInstitute/ruRoberta-large-paraphrase-v1')

def get_similarity(text1, text2):
    '''Cross-Encoder similarity thanks to Skolkovo fine-tuners'''
    with torch.inference_mode():
        batch = tokenizer(
            text1, text2, 
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

In [73]:
def bm25_ensemble_with_crossenc_answer(query, client, emb_func, bm25_n_results=10, cr_enc_n_results=2, limit_knn=70, knn_docs_window=3):
    res = _clickhouse_query_window(query, client, emb_func, limit_knn=limit_knn, docs_window=knn_docs_window)

    all_links = []
    all_docs = []
    all_pages = []

    for row in res:
        all_links.append(row[2])
        all_docs.append(row[3])
        all_pages.append(row[4])

    tokenized_corpus = [word_tokenize(doc, language='russian') for doc in all_docs]

    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = word_tokenize(query, language='russian')

    doc_scores = bm25.get_scores(tokenized_query)

    if all(doc_scores == 0):
        return ['Все найденные через KNN документы не имеют ничего общего к запросу по мнению bm25']
    
    bm25_answer = []
    args = np.argsort(doc_scores, axis=0)

    for i in range(1, bm25_n_results+1):
        bm25_answer.append(res[args[-i]])
        
    crossenc_answer = []
    for p in range(bm25_n_results):
        crossenc_answer += [get_similarity(query, bm25_answer[p][3])]
    

    final_ans = []
    args_cr = np.flipud(np.argsort(crossenc_answer, axis=0))

    for f in range(cr_enc_n_results):
        final_ans.append(bm25_answer[args_cr[f]])
    
    return final_ans #knn может вернуть не n_results, а больше, если дистанции в точности равны!

In [43]:
def bm25_ensemble(query, client, emb_func, bm25_n_results=4, limit_knn=70, knn_docs_window=3, table='index_texts_clean'):
    res = _clickhouse_query_window(query, client, emb_func, limit_knn=limit_knn, docs_window=knn_docs_window, table=table)

    all_links = []
    all_docs = []
    all_pages = []

    for row in res:
        all_links.append(row[2])
        all_docs.append(row[3])
        all_pages.append(row[4])

    tokenized_corpus = [word_tokenize(doc, language='russian') for doc in all_docs]

    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = word_tokenize(query, language='russian')

    doc_scores = bm25.get_scores(tokenized_query)

    if all(doc_scores == 0):
        return ['Все найденные через KNN документы не имеют ничего общего к запросу по мнению bm25']
    
    answer = []
    args = np.argsort(doc_scores, axis=0)

    for i in range(1, bm25_n_results+1):
        answer.append(res[args[-i]])

    return answer #knn может вернуть не n_results, а больше, если дистанции в точности равны!

In [82]:
def knn_plus_cross(query, client, emb_func, cr_enc_n_results=2, limit_knn=5, knn_docs_window=2):
    res = _clickhouse_query_window(query, client, emb_func, limit_knn=limit_knn, docs_window=knn_docs_window)

    crossenc_answer = []
    for p in range(len(res)):
        crossenc_answer += [get_similarity(query, res[p][3])]

    final_ans = []
    args_cr = np.flipud(np.argsort(crossenc_answer, axis=0))

    for f in range(cr_enc_n_results):
        final_ans.append(res[args_cr[f]])
    
    return final_ans #knn может вернуть не n_results, а больше, если дистанции в точности равны!

In [44]:
def prep_query(query):
    temp = [num2text.num2text(int(word)) if word.isdigit() else word for word in nltk.word_tokenize(query, language='russian')]

    sentence = ''
    for word in temp:
        if word not in ',,.?!:)»':
            sentence += " " + word

    return sentence

In [69]:
query=prep_query('коллекционные монеты за 2021 год.')
query

' коллекционные монеты за две тысячи двадцать один год'

In [83]:
res = knn_plus_cross(query, client, emb_func, cr_enc_n_results=2, limit_knn=3, knn_docs_window=1)
for a in res:
    print(a)

KeyboardInterrupt: 

In [ ]:
res = _clickhouse_query_l2(query=query, client=client, emb_func=emb_func, limit=10)
for a in res:
    print(a)

In [84]:
res = bm25_ensemble_with_crossenc_answer(query, client, emb_func, bm25_n_results=10, cr_enc_n_results=3, limit_knn=100, knn_docs_window=3)
for a in res:
    print(a)

10
[0.004322025924921036, 0.008001787588000298, 0.01196745689958334, 0.001486657653003931, 0.01365045178681612, 0.0015692254528403282, 0.0016374937258660793, 0.002558761974796653, 0.0026483244728296995, 0.0009159505716525018]
[4 2 1 0 8 7 6 5 3 9]
(130, 1, 'https://cbr.ru//cash_circulation/coins/2rub/', ' — стилизованный растительный орнамент в виде изогнутой ветви. Изменение аверса (2002 г.) Номинал 2 рубля Диаметр 23,00 мм Толщина 1,80 мм Масса 5,10 г Боковая поверхность (гурт): 84 рифления, разделенные на 12 равных участков, чередующихся с 12 гладкими участками Материал: медно-никелевый сплав Геометрические и физические параметры монеты, а также оформление реверса сохранены без изменений. Монета номиналом 2 рубля, изготавливавшаяся ранее с первоначальным вариантом оформления аверса, продолжает оставаться законным средством наличного платежа на территории Российской Федерации. Аверс в центре — эмблема Банка России (двуглавый орел с опущенными крыльями, под ним — надпись полукругом: «

In [46]:
res = _clickhouse_query_l2(query=query, client=client, emb_func=emb_func, limit=10)
for a in res:
    print(a)
#res = bm25_ensemble('редкие чеканные коллекционные монеты', client, emb_func, bm25_n_results=4, limit_knn=70, knn_docs_window=3, table='index_texts_clean')

(130, 3, ' разновидности монеты соответствуют ранее выпущенной монете из медно-никелевого сплава номиналом 2 рубля образца 1997 года с разновидностью аверса 2002 года. Монеты, изготавливавшиеся ранее из медно-никелевого сплава, продолжают оставаться законным средством наличного платежа на территории Российской Федерации. Изменение аверса (2016 г.) Номинал 2 рубля Диаметр 23,00 мм Толщина 1,80 мм Масса 5,00 г Боковая поверхность (гурт): 84 рифления, разделенные на 12 равных участков, чередующихся с 12 гладкими участками Материал: сталь с никелевым гальванопокрытием Массогабаритные параметры и физические характеристики монеты, а также оформление реверса сохранены без изменений. Монеты номиналом 2 рубля образца 1997 года, изготавливавшиеся ранее с предыдущими вариантами оформления аверса, продолжают оставаться законным средством наличного платежа на территории Российской Федерации. Аверс в центре — изображение Государственного герба Российской Федерации, над ним надпись полукругом: «РОССИ

In [98]:
res = _clickhouse_query_l2('сбербанк', client, emb_func, limit=100)
for a in res:
    print(a)

(13, 0, ' ЦЕНТРАЛЬНЫЙ БАНК РОССИЙСКОЙ ФЕДЕРАЦИИ (БАНК РОССИИ) 107016, Москва, ул. Неглинная, 12. к. В www.cbr.ru тел.: (499) 300-30-00, 8 (800) 300-30-00 От [ REGNUMDATESTAMP ] на от Информационное письмо Банка России об особенностях формирования резервов при кредитовании субъектов малого и среднего предпринимательства Структурным подразделениям Банка России (по списку рассылки) Кредитным организациям В целях снижения регуляторной нагрузки на кредитные организации (далее – КО) и поддержания доступности кредитования субъектов малого и среднего предпринимательства1 (далее – заемщики) Банк России информирует о следующем. 1. В рамках применения Положения Банка России № 590-П2 и Положения Банка России № 611-П3 при классификации ссуд, прочих активов и условных обязательств кредитного характера, возникших из условий кредитных договоров, заключенных с заемщиками до дня призыва на военную службу по мобилизации в Вооруженные Силы Российской Федерации единственного участника4 заемщика и по 31 дек

In [35]:
res = client.query(f'''
        SELECT distinct ID
        FROM 
            index_texts_clean
        order by ID asc
        limit 10''')

res = res.result_rows

with open('rows.txt', 'w') as f:
    for row in res:
        f.write(f'{row[0]} \n')

In [180]:
print(list(range(18649, 18654)))

[18649, 18650, 18651, 18652, 18653]


In [9]:
errors = [174, 202, 203, 204, 1048, 1049, 1050, 1051, 1052, 1053, 1162, 1163, 1164, 1165, 1166, 1570, 1571, 1572, 1602, 1604, 3010, 3017, 3024, 3033, 3312, 3390, 3397, 3409,
          3411, 3516, 3643, 3650, 3670, 3674, 3783, 3835, 3851, 3857, 
          3867, 3870, 3877, 3880, 3882, 3884, 3887, 3890, 3937, 3942, 3944, 3947, 3950, 3955, 
          3981, 3983, 3989, 4001, 4004, 4011, 4015, 4019, 4024, 4030, 4042, 4045, 4052, 4056, 4059, 4062, 4107, 4116, 4132,
          4154, 4229, 4316, 4544, 4549, 4580, 4581, 4598, 4609, 4683, 4736, 4737, 4780, 4826, 4864, 4865, 4866, 4886,
          4918, 4919, 4920, 4958, 4981, 5039, 5069, 5308, 5413, 5449, 5478, 5495, 5496, 5549, 5550, 5624, 5641, 5742,
          6517, 13742, 13764, 13786, 13787, 28752, 28787, 28814, 28848, 28902, 28925, 26545, 26546, 26600, 26653, 26710,
          26767, 26818, 26871, 26927, 26981, 27736, 27792, 27953, 21789, 23496, 15047, 15194, 15267, 15398, 15465, 15526,
          15585, 15645, 15764, 16003, 16062, 16121, 16528, 16759, 16823, 16919, 16972, 16990, 17008, 17126, 18109, 18388, 
          18322, 18323, 18324, 18325, 18326, 18327, 18328, 18329, 18330, 18331, 18332, 18333, 18334 ,
          18343, 18388, 18422, 18595, 18596, 18597, 18598, 18599, 18600, 18601, 18602, 18603, 18604, 18605, 
          18649, 18650, 18651, 18652, 18653, 18821, 19039, 19041, 19597, 19598, 20009, 20574, 20576, 20580, 20585]

In [192]:
max(errors)

28925

In [102]:
window=5

window_sql_query = ''
for i, row in enumerate(res):
    #ids_with_chunks[row[0]].extend(get_window_range(row[1], window))
    if i > 0:
        window_sql_query += ' UNION DISTINCT '
    window_sql_query += f'''SELECT * FROM index_texts WHERE ID = {row[0]} and chunk_id in {tuple(get_window_range(row[1], window))}'''

result = client.query(window_sql_query)
for row in result.result_rows:
    print(row)

(2281, 0, 'https://cbr.ru//Content/Document/File/145782/coins_cbr_2022_pr.pdf', ' СЕРИЯ «ИСТОРИЧЕСКИЕ СОБЫТИЯ» HISTORICAL EVENTS SERIES', 0, [0.04344139248132706, -0.013093809597194195, -0.02887667343020439, -0.04186256602406502, 0.03471691533923149, -0.029531968757510185, -0.02673668973147869, 0.10153445601463318, 0.09412192553281784, -0.021929247304797173, 0.025504300370812416, 0.02842095121741295, -0.03722936660051346, -0.01014799065887928, -0.015470435842871666, -0.02467544935643673, -0.03324734792113304, 0.04855277016758919, -0.0014074755599722266, 0.009713017381727695, 0.03557945415377617, -0.0164401326328516, -0.06035260111093521, -0.025361690670251846, -0.040080733597278595, -0.01564004085958004, -0.014126590453088284, -0.0271231010556221, -0.019423730671405792, -0.05740168318152428, -0.007917541079223156, 0.0005825345288030803, -0.03380168601870537, -0.037218693643808365, 0.003797924844548106, 0.031004922464489937, 0.026787137612700462, 0.04488537460565567, -0.0542599298059940

In [96]:
window_sql_query = ''
for i, (k, v) in enumerate(ids_with_chunks.items()):
    if i > 0:
        window_sql_query += ' UNION ALL '
    window_sql_query += f'''SELECT * FROM index_texts WHERE ID = {k} and chunk_id in {tuple(v)}'''
    
window_sql_query

'SELECT * FROM index_texts WHERE ID = 1952 and chunk_id in (0, 1, 2, 3, 4) UNION ALL SELECT * FROM index_texts WHERE ID = 2281 and chunk_id in (0, 1, 2, 3, 4) UNION ALL SELECT * FROM index_texts WHERE ID = 2321 and chunk_id in (0, 1, 2, 3, 4) UNION ALL SELECT * FROM index_texts WHERE ID = 2261 and chunk_id in (0, 1, 2, 3, 4) UNION ALL SELECT * FROM index_texts WHERE ID = 2267 and chunk_id in (0, 1, 2, 3, 4)'

In [98]:
result = client.query(window_sql_query)
for row in result.result_rows:
    print(row)

(1952, 0, 'https://cbr.ru//hd_base/seldomc/', ' Базы данных Котировки редких валют Курсы валют к доллару США на заданную дату Динамика курса заданной валюты к доллару США', 0, [0.02669902704656124, -0.025725161656737328, -0.040822550654411316, -0.018318643793463707, 0.013429258950054646, -0.016236204653978348, -0.017121678218245506, 0.08162964880466461, 0.07342529296875, -0.031194956973195076, 0.046875257045030594, 0.019426213577389717, -0.03225695714354515, 0.0032085629645735025, -0.005864734295755625, 0.00622314028441906, -0.038016147911548615, 0.03280050307512283, -0.01603180542588234, 0.006390362977981567, 0.032759763300418854, -4.647437162930146e-05, -0.03209870681166649, -0.02839742600917816, -0.039633143693208694, -0.027064036577939987, -0.03786955773830414, -0.03547101467847824, 0.0029091208707541227, -0.04915853217244148, -0.01963597908616066, 0.017102131620049477, 0.002590649062767625, -0.05121168866753578, -0.03817499801516533, 0.030823180451989174, 0.008691269904375076, 0.0

In [95]:
for row in result.result_rows:
    print(row)

(1952, 0, 'https://cbr.ru//hd_base/seldomc/', ' Базы данных Котировки редких валют Курсы валют к доллару США на заданную дату Динамика курса заданной валюты к доллару США', 0, [0.02669902704656124, -0.025725161656737328, -0.040822550654411316, -0.018318643793463707, 0.013429258950054646, -0.016236204653978348, -0.017121678218245506, 0.08162964880466461, 0.07342529296875, -0.031194956973195076, 0.046875257045030594, 0.019426213577389717, -0.03225695714354515, 0.0032085629645735025, -0.005864734295755625, 0.00622314028441906, -0.038016147911548615, 0.03280050307512283, -0.01603180542588234, 0.006390362977981567, 0.032759763300418854, -4.647437162930146e-05, -0.03209870681166649, -0.02839742600917816, -0.039633143693208694, -0.027064036577939987, -0.03786955773830414, -0.03547101467847824, 0.0029091208707541227, -0.04915853217244148, -0.01963597908616066, 0.017102131620049477, 0.002590649062767625, -0.05121168866753578, -0.03817499801516533, 0.030823180451989174, 0.008691269904375076, 0.0